
## Quantitative Text Analysis Lab Session: Week 12

### Topic: Finet-tuning Flair NLP with Irish Enviromental Policies 

-----

Instructor: Yen-Chieh Liao and Stefan Müller 

Date: 22 April 2024

##### __Preprocessing Data__

All Packages

In [1]:
from flair.data import Sentence
from flair.datasets import SentenceDataset
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.embeddings import TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.data import Corpus
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np
import torch
import flair
import random

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Check if GPU is Available

In [5]:
# check if GPU is available
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps' 
else:
    device = 'cpu'

print('GPU Device:',device)

GPU Device: mps


Load the Formatted Dataset

In [6]:
from datasets import DatasetDict
dataset = DatasetDict.load_from_disk('irish_environmental_policies')
train, validation, test = dataset['train'], dataset['validation'], dataset['test']

Creation of the Corpus and Label

In [8]:
from toolbox import prepare_dataset
random.seed(42)
train_dataset = prepare_dataset(train)
validation_dataset = prepare_dataset(validation)
test_dataset = prepare_dataset(test)

# train_dataset = random.sample(train_dataset, 800)
# validation_dataset = random.sample(validation_dataset, 200)
# test_dataset = random.sample(test_dataset, 200)

Creation of the Corpus and Label

In [9]:
corpus = Corpus(train=train_dataset, dev=validation_dataset, test=test_dataset)
label_dict = corpus.make_label_dictionary(label_type='label')

2025-07-29 10:11:07,743 Computing label dictionary. Progress:


0it [00:00, ?it/s]
1880it [00:00, 88661.54it/s]

2025-07-29 10:11:07,772 Dictionary created for label 'label' with 2 values: 0 (seen 1772 times), 1 (seen 108 times)


Check the Mapping from Label to Index

In [10]:
print("----- Inspecting Label Types and Frequency Distribution ------")
print("Lable Check: {}".format(label_dict))
from collections import Counter
label_counter = Counter()
for sentence in corpus.get_all_sentences():
    labels = sentence.get_labels('label')
    label_counter.update([label.value for label in labels])
for label, frequency in label_counter.items():
    print(f"Label '{label}': {frequency} times")

----- Inspecting Label Types and Frequency Distribution ------
Lable Check: Dictionary with 2 tags: 0, 1
Label '0': 2937 times
Label '1': 197 times


In [11]:
print("------ Check the Mapping Order of idx2item and item2id -------")
print("Check idx2item Mapping : {}".format(label_dict.idx2item))
print("Check item2idx Mapping : {}".format(label_dict.item2idx))

------ Check the Mapping Order of idx2item and item2id -------
Check idx2item Mapping : [b'0', b'1']
Check item2idx Mapping : {b'0': 0, b'1': 1}


In [12]:
sbert_embeddings = TransformerDocumentEmbeddings('sentence-transformers/distiluse-base-multilingual-cased-v2', fine_tune=True)

In [13]:
sbert_classifier = TextClassifier(sbert_embeddings, 
                                  label_dictionary=label_dict,
                                  label_type='label')
# sbert_classifier = sbert_classifier.to(device)

Start Trainning

In [14]:
sbert_trainer = ModelTrainer(sbert_classifier, corpus)
sbert_trainer.train('qta_flair_python_model',      
                    shuffle = True,               
                    patience=3,            
                    learning_rate=0.02,             
                    mini_batch_size=16, 
                    write_weights = True,          
                    max_epochs=3)     

2025-07-29 10:11:20,775 ----------------------------------------------------------------------------------------------------
2025-07-29 10:11:20,777 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119548, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, ou

100%|██████████| 10/10 [00:07<00:00,  1.36it/s]

2025-07-29 10:12:50,424 DEV : loss 0.12341202795505524 - f1-score (micro avg)  0.9569
2025-07-29 10:12:50,430  - 0 epochs without improvement
2025-07-29 10:12:50,431 saving best model


2025-07-29 10:12:51,506 ----------------------------------------------------------------------------------------------------
2025-07-29 10:12:59,187 epoch 2 - iter 11/118 - loss 0.11401551 - time (sec): 7.68 - samples/sec: 22.92 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:07,776 epoch 2 - iter 22/118 - loss 0.11507420 - time (sec): 16.27 - samples/sec: 21.64 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:15,315 epoch 2 - iter 33/118 - loss 0.10087184 - time (sec): 23.81 - samples/sec: 22.18 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:22,387 epoch 2 - iter 44/118 - loss 0.10967341 - time (sec): 30.88 - samples/sec: 22.80 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:30,026 epoch 2 - iter 55/118 - loss 0.11562597 - time (sec): 38.52 - samples/sec: 22.85 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:38,780 epoch 2 - iter 66/118 - loss 0.11602521 - time (sec): 47.27 - samples/sec: 22.34 - lr: 0.020000 - momentum: 0.000000
2025-07-29 10:13:46,696 epoch 

100%|██████████| 10/10 [00:00<00:00, 159.38it/s]

...

2025-07-29 10:15:40,636 DEV : loss 0.1239851713180542 - f1-score (micro avg)  0.9585
2025-07-29 10:15:40,642  - 1 epochs without improvement


2025-07-29 10:15:41,935 ----------------------------------------------------------------------------------------------------
2025-07-29 10:15:41,936 Loading model from best epoch ...


100%|██████████| 10/10 [00:07<00:00,  1.37it/s]

2025-07-29 10:15:52,859 
Results:
- F-score (micro) 0.949
- F-score (macro) 0.7366
- Accuracy 0.949

By class:
              precision    recall  f1-score   support

           0     0.9539    0.9931    0.9731       583
           1     0.8000    0.3636    0.5000        44

    accuracy                         0.9490       627
   macro avg     0.8769    0.6784    0.7366       627
weighted avg     0.9431    0.9490    0.9399       627

2025-07-29 10:15:52,861 ----------------------------------------------------------------------------------------------------


{'test_score': 0.94896331738437}